In [1]:
!pip install faiss-cpu sentence-transformers
!pip install -q transformers datasets peft accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 11.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

________________________________________________________________

________________________________________________________________

In [6]:
import os

base = '/content/drive/MyDrive/text2sql_project/datasets'

for dataset in ['chinook']:
    print(f"\nContents of {dataset.upper()} dataset:")
    folder_path = os.path.join(base, dataset)
    for root, dirs, files in os.walk(folder_path):
        level = root.replace(folder_path, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print(f"{indent}{os.path.basename(root)}/")
        for f in files:
            print(f"{indent}    {f}")


Contents of CHINOOK dataset:
chinook/
    chinook.db
    chinook_data.json
    chinook_test_30.json


In [3]:
import sqlite3
import json

# Connect to the Chinook database
conn = sqlite3.connect('/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook.db')
cursor = conn.cursor()

# Fetch schema for all tables
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

schema_dict = {}
for table in tables:
    cursor.execute(f"SELECT sql FROM sqlite_master WHERE type='table' AND name='{table}';")
    create_stmt = cursor.fetchone()[0]
    schema_dict[table] = create_stmt

In [4]:
import sqlite3

# Path to your Chinook database
db_path = '/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook.db'

# Connect to the database
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Query for all tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Display table names
print("Tables in Chinook database:")
for table in tables:
    print("-", table[0])

conn.close()

Tables in Chinook database:
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track


________________________________________________________________

________________________________________________________________

In [7]:
import json
from sentence_transformers import SentenceTransformer

# Load the Chinook dataset
with open('/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_data.json', 'r') as f:
    data = json.load(f)

# Extract a flat list of schema and example-based context
corpus = []

# Add individual CREATE TABLE statements
for table_name, create_stmt in data[0]['schema'].items():
    corpus.append(f"{create_stmt}")

# Optionally add a few example questions + SQLs as retrievable knowledge
for i, entry in enumerate(data[:25]):  # First 25 as retrieval base
    corpus.append(f"Example {i+1} - Question: {entry['question']} SQL: {entry['query']}")



In [8]:
print(corpus)

['CREATE TABLE [Album]\n(\n    [AlbumId] INTEGER  NOT NULL,\n    [Title] NVARCHAR(160)  NOT NULL,\n    [ArtistId] INTEGER  NOT NULL,\n    CONSTRAINT [PK_Album] PRIMARY KEY  ([AlbumId]),\n    FOREIGN KEY ([ArtistId]) REFERENCES [Artist] ([ArtistId]) \n\t\tON DELETE NO ACTION ON UPDATE NO ACTION\n)', 'CREATE TABLE [Artist]\n(\n    [ArtistId] INTEGER  NOT NULL,\n    [Name] NVARCHAR(120),\n    CONSTRAINT [PK_Artist] PRIMARY KEY  ([ArtistId])\n)', 'CREATE TABLE [Customer]\n(\n    [CustomerId] INTEGER  NOT NULL,\n    [FirstName] NVARCHAR(40)  NOT NULL,\n    [LastName] NVARCHAR(20)  NOT NULL,\n    [Company] NVARCHAR(80),\n    [Address] NVARCHAR(70),\n    [City] NVARCHAR(40),\n    [State] NVARCHAR(40),\n    [Country] NVARCHAR(40),\n    [PostalCode] NVARCHAR(10),\n    [Phone] NVARCHAR(24),\n    [Fax] NVARCHAR(24),\n    [Email] NVARCHAR(60)  NOT NULL,\n    [SupportRepId] INTEGER,\n    CONSTRAINT [PK_Customer] PRIMARY KEY  ([CustomerId]),\n    FOREIGN KEY ([SupportRepId]) REFERENCES [Employee] ([

In [9]:
import faiss
import numpy as np

retriever_model = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = retriever_model.encode(corpus, convert_to_numpy=True)

index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
def retrieve_context(question, top_k=3):
    query_embedding = retriever_model.encode([question], convert_to_numpy=True)
    D, I = index.search(query_embedding, top_k)
    return [corpus[i] for i in I[0]]


In [11]:
test_question = "Which customers have made purchases over $30?"
retrieved_chunks = retrieve_context(test_question)
print("\n".join(retrieved_chunks))


Example 8 - Question: Find the names and emails of customers who have made purchases over $20. SQL: SELECT DISTINCT Customer.FirstName, Customer.LastName, Customer.Email FROM Customer JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId WHERE Invoice.Total > 20;
Example 18 - Question: Show the customers who have not made any purchases. SQL: SELECT FirstName, LastName FROM Customer WHERE CustomerId NOT IN (SELECT CustomerId FROM Invoice);
Example 3 - Question: List the top 5 most purchased tracks. SQL: SELECT Track.Name, COUNT(*) as Purchases FROM InvoiceLine JOIN Track ON InvoiceLine.TrackId = Track.TrackId GROUP BY Track.Name ORDER BY Purchases DESC LIMIT 5;


In [12]:
def build_rag_prompt(question, use_full_schema=True):
    if use_full_schema:
        schema_text = "\n".join([f"{k}: {v}" for k, v in data[0]['schema'].items()])
        context = schema_text
    else:
        context = "\n".join(retrieve_context(question))
    return f"""You are a SQL expert. Use the following context to write an accurate SQL query.

### Context:
{context}

### Question:
{question}

### SQL:"""


________________________________________________________________

________________________________________________________________

In [13]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch

model_name = "codellama/CodeLlama-7b-Instruct-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",        # lets Colab assign it to GPU
    torch_dtype="auto"
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/646 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Device set to use cuda:0


In [14]:
test_question = "Provide a query that includes the purchased track name and artist name with each invoice line item."
prompt = build_rag_prompt(test_question, use_full_schema=True)

# Run inference
output = generator(prompt, max_new_tokens=256, do_sample=False)[0]["generated_text"]

# Clean result (strip original prompt)
generated_sql = output.replace(prompt, "").strip()
#if "```" in generated_sql:
#    generated_sql = generated_sql.replace("```sql", "").replace("```", "").strip()

print("Generated SQL:\n", generated_sql)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated SQL:
 ```sql
SELECT
    i.InvoiceId,
    i.InvoiceDate,
    i.Total,
    t.Name AS TrackName,
    a.Name AS ArtistName
FROM
    Invoice i
    INNER JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId
    INNER JOIN Track t ON il.TrackId = t.TrackId
    INNER JOIN Album a ON t.AlbumId = a.AlbumId
    INNER JOIN Artist ar ON a.ArtistId = ar.ArtistId
```

### Explanation:

The query joins the Invoice table to the InvoiceLine table to get the invoice line items. It then joins the Track table to the InvoiceLine table to get the track information. Finally, it joins the Album table to the Track table to get the artist information.

### Example:

![](https://assets.aaonline.io/Module-Web_Services/images/sql_query_result.png)

### Hint:

* Use the INNER JOIN keyword to join tables.
* Use the ON clause to specify


________________________________________________________________

________________________________________________________________

In [15]:
# Loop through all 88 samples and generate SQL using CodeLlama + RAG
generated_results = []

for i, entry in enumerate(data):
    question = entry["question"]
    ground_truth = entry["query"]

    try:
        # Build RAG-enhanced prompt with schema
        prompt = build_rag_prompt(question, use_full_schema=True)

        # Generate SQL
        output = generator(prompt, max_new_tokens=256, do_sample=False)[0]["generated_text"]

        # Extract only the SQL part
        generated_sql = output.replace(prompt, "").strip()

        # Remove markdown SQL blocks if they exist
        if "```" in generated_sql:
            generated_sql = generated_sql.replace("```sql", "").replace("```", "").strip()

        # Store result
        generated_results.append({
            "index": i,
            "question": question,
            "ground_truth": ground_truth,
            "generated_sql": generated_sql
        })

        print(f"[{i+1}/100] Success - Q: {question}")

    except Exception as e:
        print(f"[{i+1}/100] Failed - {question}\nError: {str(e)}")
        generated_results.append({
            "index": i,
            "question": question,
            "ground_truth": ground_truth,
            "generated_sql": None,
            "error": str(e)
        })


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[1/100] Success - Q: What are the names of all albums by AC/DC?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[2/100] Success - Q: Which customers are from Brazil?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

[85/100] Success - Q: Show tracks that were purchased more than 10 times.


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[86/100] Success - Q: Which invoices have more than $40 in total and less than 5 items?


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[87/100] Success - Q: Find the number of albums per artist.
[88/100] Success - Q: List all customers who purchased Jazz tracks and live in Canada.


In [16]:
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql.json"

with open(output_path, 'w') as f:
    json.dump(generated_results, f, indent=4)

print("Results saved to:", output_path)


Results saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql.json


________________________________________________________________

________________________________________________________________

In [17]:
def normalize_sql(sql: str):
    if sql is None:
        return None
    return " ".join(sql.strip().replace("\n", " ").lower().split())

In [18]:
import sqlite3

db_path = '/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

evaluated_results = []

for item in generated_results:
    question = item["question"]
    gold_sql = normalize_sql(item["ground_truth"])
    pred_sql = normalize_sql(item["generated_sql"])

    em = (gold_sql == pred_sql)

    try:
        # Run ground truth SQL
        cursor.execute(item["ground_truth"])
        gold_result = cursor.fetchall()

        # Run generated SQL
        cursor.execute(item["generated_sql"])
        pred_result = cursor.fetchall()

        # Compare unordered result sets
        ex = sorted(gold_result) == sorted(pred_result)
    except Exception as e:
        ex = False

    evaluated_results.append({
        "question": question,
        "exact_match": em,
        "execution_match": ex,
        "ground_truth": item["ground_truth"],
        "generated_sql": item["generated_sql"]
    })

conn.close()


In [19]:
total = len(evaluated_results)
em_score = sum(r["exact_match"] for r in evaluated_results) / total
ex_score = sum(r["execution_match"] for r in evaluated_results) / total

print(f"\nEvaluation Complete")
print(f"Exact Match Accuracy: {em_score:.2%}")
print(f"Execution Accuracy:   {ex_score:.2%}")



Evaluation Complete
Exact Match Accuracy: 0.00%
Execution Accuracy:   0.00%


________________________________________________________________

________________________________________________________________

In [20]:
import json
import os

input_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql.json"
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_fixed.json"

# SQL NORMALIZATION FUNCTION
def normalize_sql(sql):
    if not sql:
        return ""
    # Remove newlines and markdown-style notes (e.g., ### SQL)
    sql = sql.replace("\n", " ").replace("[", "").replace("]", "")
    sql = sql.split("###")[0].strip()  # Remove any trailing explanations
    # Remove duplicate spaces
    return " ".join(sql.strip().split())

# CLEANING FUNCTION
def clean_entry(entry):
    if "ground_truth" in entry:
        entry["ground_truth"] = normalize_sql(entry["ground_truth"])
    if "generated_sql" in entry:
        entry["generated_sql"] = normalize_sql(entry["generated_sql"])
    return entry

# LOAD, CLEAN, AND SAVE
if not os.path.exists(input_path):
    raise FileNotFoundError(f"File not found: {input_path}")

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# Apply cleaning to all entries
cleaned_data = [clean_entry(entry) for entry in data]

# Save the cleaned output
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=4)

print(f"Cleaned SQL file saved to: {output_path}")


Cleaned SQL file saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_fixed.json


In [21]:
import json
import sqlite3
import os

# CONFIG
input_json = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_fixed.json"
db_path = "/content/chinook.db"  # Must copy from Drive if not in /content
output_json = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results.json"

# COPY DB LOCALLY if needed
drive_db_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook.db"
if not os.path.exists(db_path):
    !cp "{drive_db_path}" "{db_path}"

# SQL NORMALIZATION FUNCTION
def normalize_sql(sql: str) -> str:
    if not sql:
        return ""
    sql = sql.replace("\n", " ").replace("[", "").replace("]", "")
    return " ".join(sql.lower().strip().split())

# LOAD DATA
with open(input_json, "r") as f:
    data = json.load(f)

# CONNECT TO DATABASE
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# EVALUATION
em_count, ex_count = 0, 0
total = len(data)

for row in data:
    gt_sql = row.get("ground_truth", "")
    gen_sql = row.get("generated_sql", "")

    # Exact Match (normalized)
    row["exact_match"] = normalize_sql(gt_sql) == normalize_sql(gen_sql)
    if row["exact_match"]:
        em_count += 1

    # Execution Match
    try:
        cursor.execute(gt_sql)
        gt_result = cursor.fetchall()

        cursor.execute(gen_sql)
        gen_result = cursor.fetchall()

        row["execution_match"] = sorted(gt_result) == sorted(gen_result)
    except Exception as e:
        row["execution_match"] = False
        row["execution_error"] = str(e)

    if row["execution_match"]:
        ex_count += 1

conn.close()

# SAVE EVALUATED RESULTS
with open(output_json, "w") as f:
    json.dump(data, f, indent=4)

# PRINT SUMMARY
print(f"Evaluation Complete")
print(f"Exact Match Accuracy:    {em_count / total:.2%}")
print(f"Execution Match Accuracy: {ex_count / total:.2%}")
print(f"Results saved to: {output_json}")

Evaluation Complete
Exact Match Accuracy:    3.41%
Execution Match Accuracy: 31.82%
Results saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results.json


In [22]:
import json

# CONFIG
input_json = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results.json"
failures_output_json = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_failed_cases.json"

# LOAD EVALUATED RESULTS
with open(input_json, "r") as f:
    data = json.load(f)

# FILTER FAILING CASES
failing_cases = [
    entry for entry in data
    if not entry.get("exact_match", False) or not entry.get("execution_match", False)
]

# SAVE TO NEW FILE
with open(failures_output_json, "w") as f:
    json.dump(failing_cases, f, indent=4)

# PRINT SUMMARY
print(f"Total failing cases: {len(failing_cases)}")
print(f"Saved to: {failures_output_json}")


Total failing cases: 85
Saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_eval_failed_cases.json


In [23]:
import json
import pandas as pd

# CONFIG
input_json = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results.json"
output_csv = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_failed_cases.csv"

# LOAD DATA
with open(input_json, "r") as f:
    data = json.load(f)

# FILTER FAILURES
failing = [
    {
        "question": d.get("question", ""),
        "ground_truth": d.get("ground_truth", ""),
        "generated_sql": d.get("generated_sql", ""),
        "exact_match": d.get("exact_match", False),
        "execution_match": d.get("execution_match", False),
        "execution_error": d.get("execution_error", "")
    }
    for d in data if not d.get("exact_match", False) or not d.get("execution_match", False)
]

# CONVERT TO CSV
df = pd.DataFrame(failing)
df.to_csv(output_csv, index=False)

print(f"Failing cases saved as CSV to: {output_csv}")


Failing cases saved as CSV to: /content/drive/MyDrive/text2sql_project/outputs/chinook_eval_failed_cases.csv


________________________________________________________________

________________________________________________________________

In [24]:
import json

# FILE PATHS
input_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_data.json"
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_finetune.jsonl"

# LOAD CHINOOK DATA
with open(input_path, "r", encoding="utf-8") as f:
    chinook_data = json.load(f)

# CONVERT TO INSTRUCTION FORMAT
finetune_data = []
for entry in chinook_data:
    finetune_data.append({
        "instruction": "Write an SQL query for the Chinook database.",
        "input": entry["question"],
        "output": entry["query"]
    })

# SAVE AS JSONL FILE
with open(output_path, "w", encoding="utf-8") as f:
    for item in finetune_data:
        f.write(json.dumps(item) + "\n")

print(f"Fine-tuning file saved to: {output_path}")


Fine-tuning file saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_finetune.jsonl


________________________________________________________________

________________________________________________________________

In [25]:
from transformers import set_seed
set_seed(42)

In [26]:
from datasets import load_dataset
from transformers import AutoTokenizer
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling
import torch

# Load tokenizer
model_name = "codellama/CodeLlama-7b-Instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True, padding_side="right", truncation_side="right")
tokenizer.pad_token = tokenizer.eos_token

In [27]:
# Load dataset from Drive
finetune_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_finetune.jsonl"

dataset = load_dataset("json", data_files={"train": finetune_path}, split="train")

# Tokenization
def tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n"
    full_text = prompt + example["output"]
    return tokenizer(full_text, truncation=True, padding="max_length", max_length=512)

tokenized_dataset = dataset.map(tokenize)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/88 [00:00<?, ? examples/s]

In [28]:
from peft import TaskType

# Load full 16-bit model (not quantized)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

# Wrap model with PEFT (LoRA)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 4,194,304 || all params: 6,742,740,992 || trainable%: 0.0622


In [29]:
output_dir = "/content/drive/MyDrive/text2sql_project/models/codellama_chinook_lora"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    logging_steps=10,
    save_steps=500,
    save_total_limit=2,
    learning_rate=2e-4,
    fp16=True,
    report_to="none",
    logging_dir=f"{output_dir}/logs"
)

# Use padding collator for causal LM
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=training_args,
    data_collator=data_collator,
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [30]:
trainer.train()

Step,Training Loss
10,1.661900
20,0.733500
30,0.521700
40,0.478000
50,0.458600


TrainOutput(global_step=50, training_loss=0.7707512950897217, metrics={'train_runtime': 88.0319, 'train_samples_per_second': 9.996, 'train_steps_per_second': 0.568, 'total_flos': 1.4948782521188352e+16, 'train_loss': 0.7707512950897217, 'epoch': 8.363636363636363})

In [31]:
model.save_pretrained("/content/drive/MyDrive/text2sql_project/models/codellama_chinook_lora")

________________________________________________________________

___________________________________________________________

In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

# Paths
base_model = "codellama/CodeLlama-7b-Instruct-hf"
lora_path = "/content/drive/MyDrive/text2sql_project/models/codellama_chinook_lora"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model)
tokenizer.pad_token = tokenizer.eos_token  # Ensure padding works

# Load base model + LoRA
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16, device_map="auto")
model = PeftModel.from_pretrained(model, lora_path)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
import torch

def generate_sql(question, model, tokenizer, max_tokens=256):
    model.eval()
    device = model.device

    prompt = f"""### Instruction:
Write an SQL query for the Chinook database.

### Input:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.replace(prompt, "").strip()


In [34]:
test_question = "Which customers have spent more than $50 total?"
sql = generate_sql(test_question, model, tokenizer)
print("Generated SQL:\n", sql)

Generated SQL:
 SELECT Customer.FirstName, Customer.LastName FROM Customer JOIN Invoice ON Customer.CustomerId = Invoice.CustomerId GROUP BY Customer.CustomerId HAVING SUM(Invoice.Total) > 50;


________________________________________________

________________________________________________

In [35]:
import json
from tqdm import tqdm

# Paths
input_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_data.json"
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_finetuned.json"

# Load Chinook questions
with open(input_path, "r") as f:
    chinook_data = json.load(f)

# SQL Generation Function (reusing from earlier)
def generate_sql(question, model, tokenizer, max_tokens=256):
    model.eval()
    device = model.device

    prompt = f"""### Instruction:
Write an SQL query for the Chinook database.

### Input:
{question}

### Response:
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded.replace(prompt, "").strip()

# Batch Generate SQL
results = []
for entry in tqdm(chinook_data, desc="Generating SQL"):
    question = entry["question"]
    generated_sql = generate_sql(question, model, tokenizer)
    results.append({
        "question": question,
        "generated_sql": generated_sql
    })

# Save to JSON
with open(output_path, "w") as f:
    json.dump(results, f, indent=4)

print(f"All queries generated and saved to: {output_path}")


Generating SQL: 100%|██████████| 88/88 [1:13:15<00:00, 49.95s/it]


All queries generated and saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_finetuned.json


___________________________________________________________

___________________________________________________________

In [36]:
import sqlite3
import json
import re

gen_sql_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_finetuned.json"
ground_truth_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_data.json"
db_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook.db"
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results_finetuned.json"

# Load data
with open(gen_sql_path) as f:
    gen_data = json.load(f)

with open(ground_truth_path) as f:
    gt_data = json.load(f)

# Normalize and match entries
correct_exact = 0
correct_exec = 0
results = []

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

def normalize(sql):
    sql = sql.strip().lower()
    sql = re.sub(r"\s+", " ", sql)  # collapse whitespace
    return sql

for gen, gt in zip(gen_data, gt_data):
    q = gen["question"]
    pred = normalize(gen["generated_sql"])
    gold = normalize(gt["query"])

    exact_match = pred == gold
    exec_match = False

    try:
        cursor.execute(pred)
        gen_result = cursor.fetchall()

        cursor.execute(gold)
        gt_result = cursor.fetchall()

        exec_match = gen_result == gt_result
    except:
        exec_match = False

    results.append({
        "question": q,
        "generated_sql": pred,
        "gold_sql": gold,
        "exact_match": exact_match,
        "execution_match": exec_match
    })

    correct_exact += int(exact_match)
    correct_exec += int(exec_match)

conn.close()

# Save evaluation
with open(output_path, "w") as f:
    json.dump(results, f, indent=4)

total = len(gt_data)
print("Evaluation Complete")
print(f"Exact Match Accuracy:    {correct_exact / total * 100:.2f}%")
print(f"Execution Match Accuracy: {correct_exec / total * 100:.2f}%")
print(f"Results saved to: {output_path}")


Evaluation Complete
Exact Match Accuracy:    36.36%
Execution Match Accuracy: 60.23%
Results saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results_finetuned.json


____________________________________________________________________________

____________________________________________________________________________

____________________________________________________________________________

____________________________________________________________________________

In [37]:
import os
import json

# Load test set and schema
test_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_test_30.json"
schema_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_data.json"
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_prompts_testset.json"

# Load test questions
with open(test_path, "r") as f:
    test_set = json.load(f)

# Load schema from the first entry
with open(schema_path, "r") as f:
    schema_dict = json.load(f)[0]["schema"]

# Build prompts
def build_prompt(question):
    schema_text = "\n".join([f"{k}: {v}" for k, v in schema_dict.items()])
    return f"""You are a SQL expert. Use the following context to write an accurate SQL query.

### Context:
{schema_text}

### Question:
{question}

### SQL:"""

# Generate prompts
test_prompts = [
    {
        "question": entry["question"],
        "query": entry["query"],
        "prompt": build_prompt(entry["question"])
    }
    for entry in test_set
]

# Save for inference
with open(output_path, "w") as f:
    json.dump(test_prompts, f, indent=4)

print(f"Prompts saved to: {output_path}")



Prompts saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_prompts_testset.json


____________________________________________________________________________

____________________________________________________________________________

In [1]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Load fine-tuned model
base_model = "codellama/CodeLlama-7b-Instruct-hf"
lora_path = "/content/drive/MyDrive/text2sql_project/models/codellama_chinook_lora"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto"
)
model = PeftModel.from_pretrained(model, lora_path)
model.eval()

# Load prompts
with open("/content/drive/MyDrive/text2sql_project/outputs/chinook_prompts_testset.json", "r") as f:
    prompts = json.load(f)

# Generate SQL
generated_outputs = []
for item in prompts:
    prompt = item["prompt"]
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_sql = decoded.replace(prompt, "").strip().replace("\n", " ").strip("`sql").strip()

    generated_outputs.append({
        "question": item["question"],
        "ground_truth": item["query"],
        "generated_sql": generated_sql
    })

# Save output
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_testset.json"
with open(output_path, "w") as f:
    json.dump(generated_outputs, f, indent=4)

print(f"Finished! SQLs saved to: {output_path}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Finished! SQLs saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_testset.json


___________________________________________________________

___________________________________________________________

In [2]:
import sqlite3
import json

# Set paths
db_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook.db"
input_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_testset.json"
eval_output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results_testset.json"

# Load generated SQLs
with open(input_path, "r") as f:
    generated = json.load(f)

# Evaluate each query
exact_match = 0
exec_match = 0
total = len(generated)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for entry in generated:
    pred_sql = entry["generated_sql"].strip().lower().rstrip(";")
    true_sql = entry["ground_truth"].strip().lower().rstrip(";")

    # Exact match evaluation
    if pred_sql == true_sql:
        exact_match += 1

    # Execution match evaluation
    try:
        cursor.execute(pred_sql)
        pred_result = cursor.fetchall()

        cursor.execute(true_sql)
        true_result = cursor.fetchall()

        if pred_result == true_result:
            exec_match += 1
    except Exception as e:
        entry["error"] = str(e)

conn.close()

# Compute and save metrics
eval_metrics = {
    "Exact Match Accuracy": round(100 * exact_match / total, 2),
    "Execution Match Accuracy": round(100 * exec_match / total, 2)
}

with open(eval_output_path, "w") as f:
    json.dump({"results": generated, "metrics": eval_metrics}, f, indent=4)

print("Evaluation Complete")
print("Exact Match Accuracy:", eval_metrics["Exact Match Accuracy"], "%")
print("Execution Match Accuracy:", eval_metrics["Execution Match Accuracy"], "%")
print("Results saved to:", eval_output_path)


Evaluation Complete
Exact Match Accuracy: 0.0 %
Execution Match Accuracy: 23.33 %
Results saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results_testset.json


___________________________________________________________

___________________________________________________________

In [4]:
import json
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Load 88 fine-tune examples and Chinook schema
with open("/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_data.json", "r") as f:
    finetune_data = json.load(f)

schema_dict = finetune_data[0]["schema"]

# Build RAG Corpus
corpus = []

# Add schema as context chunks
corpus.append("Schema:")
for table_name, create_stmt in schema_dict.items():
    corpus.append(f"{create_stmt}")

# Add 50 example question-query pairs as retrievable examples
for i, entry in enumerate(finetune_data[:50]):
    corpus.append(f"Example {i+1} - Question: {entry['question']} SQL: {entry['query']}")

# Embed the corpus
model = SentenceTransformer("all-MiniLM-L6-v2")
corpus_embeddings = model.encode(corpus, convert_to_numpy=True)

# Build FAISS index
index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

def retrieve_context(question, top_k=3):
    q_emb = model.encode([question], convert_to_numpy=True)
    D, I = index.search(q_emb, top_k)
    return [corpus[i] for i in I[0]]

# Load the 30 test questions
with open("/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook_test_30.json", "r") as f:
    test_data = json.load(f)

# Create prompts using RAG
rag_prompts = []

for row in test_data:
    question = row["question"]
    query = row["query"]
    context_chunks = retrieve_context(question, top_k=4)

    prompt = f"""You are a SQL expert. Use the following context to write an accurate SQL query.

### Context:
{chr(10).join(context_chunks)}

### Question:
{question}

### SQL:"""

    rag_prompts.append({
        "question": question,
        "query": query,
        "prompt": prompt
    })

# Save RAG-enhanced prompts
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_prompts_testset_rag.json"
with open(output_path, "w") as f:
    json.dump(rag_prompts, f, indent=4)

print(f"RAG-enhanced prompts saved to: {output_path}")


RAG-enhanced prompts saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_prompts_testset_rag.json


In [5]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# Load Fine-Tuned Model
base_model = "codellama/CodeLlama-7b-Instruct-hf"
lora_model_path = "/content/drive/MyDrive/text2sql_project/models/codellama_chinook_lora"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(base_model, torch_dtype=torch.float16, device_map="auto")
model = PeftModel.from_pretrained(model, lora_model_path)
model.eval()

# Load RAG-enhanced prompts
with open("/content/drive/MyDrive/text2sql_project/outputs/chinook_prompts_testset_rag.json", "r") as f:
    rag_prompts = json.load(f)

# Generate SQL
generated_outputs = []

for item in rag_prompts:
    prompt = item["prompt"]
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated_sql = decoded.replace(prompt, "").strip().replace("\n", " ").strip("`sql").strip()

    generated_outputs.append({
        "question": item["question"],
        "ground_truth": item["query"],
        "generated_sql": generated_sql
    })

# Save Output
output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_testset_rag.json"
with open(output_path, "w") as f:
    json.dump(generated_outputs, f, indent=4)

print(f"SQL generation complete. Saved to: {output_path}")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SQL generation complete. Saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_testset_rag.json


In [6]:
import sqlite3
import json

# Paths
db_path = "/content/drive/MyDrive/text2sql_project/datasets/chinook/chinook.db"
input_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_generated_sql_testset_rag.json"
eval_output_path = "/content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results_testset_rag.json"

# Load predictions
with open(input_path, "r") as f:
    generated = json.load(f)

# Evaluate
exact_match = 0
exec_match = 0
total = len(generated)

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

for entry in generated:
    pred_sql = entry["generated_sql"].strip().lower().rstrip(";")
    true_sql = entry["ground_truth"].strip().lower().rstrip(";")

    # Exact Match
    if pred_sql == true_sql:
        exact_match += 1

    # Execution Match
    try:
        cursor.execute(pred_sql)
        pred_result = cursor.fetchall()

        cursor.execute(true_sql)
        true_result = cursor.fetchall()

        if pred_result == true_result:
            exec_match += 1
    except Exception as e:
        entry["error"] = str(e)

conn.close()

# Metrics
metrics = {
    "Exact Match Accuracy": round(100 * exact_match / total, 2),
    "Execution Match Accuracy": round(100 * exec_match / total, 2)
}

# Save results
with open(eval_output_path, "w") as f:
    json.dump({"results": generated, "metrics": metrics}, f, indent=4)

print("RAG Evaluation Complete")
print("Exact Match Accuracy:", metrics["Exact Match Accuracy"], "%")
print("Execution Match Accuracy:", metrics["Execution Match Accuracy"], "%")
print("Results saved to:", eval_output_path)


RAG Evaluation Complete
Exact Match Accuracy: 16.67 %
Execution Match Accuracy: 40.0 %
Results saved to: /content/drive/MyDrive/text2sql_project/outputs/chinook_eval_results_testset_rag.json


___________________________________________________________

___________________________________________________________